In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
np.set_printoptions(threshold=np.inf, suppress=True)
import matplotlib.pyplot as plt

Loading data

In [2]:
sonar_data = pd.read_csv('dataset/sonar.all-data', header=None)
print(sonar_data.head())
print(sonar_data.shape)
print(sonar_data[60].value_counts())

       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   R  
1  0.0052  0.0044   R  
2  0.0095  0.0078   

In [3]:
sonar_data.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [4]:
# replacnutie R za 0 a M za 1
# R - Rock M - Mina
sonar_data[60] = sonar_data[60].replace(['R', 'M'], [0, 1])
sonar_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


Data normalization

In [5]:
normalized_dfMinMax = sonar_data.copy()
for x in range(60):
    normalized_dfMinMax[x] = MinMaxScaler().fit_transform(np.array(normalized_dfMinMax[x]).reshape(-1,1))

normalized_dfMinMax

normalized_df = sonar_data.copy()
for x in range(60):
    normalized_df[x] = StandardScaler().fit_transform(np.array(normalized_df[x]).reshape(-1,1))

normalized_df


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,-0.399551,-0.040648,-0.026926,-0.715105,0.364456,-0.101253,0.521638,0.297843,1.125272,0.021186,...,-1.115432,-0.597604,0.680897,-0.295646,1.481635,1.763784,0.069870,0.171678,-0.658947,0
1,0.703538,0.421630,1.055618,0.323330,0.777676,2.607217,1.522625,2.510982,1.318325,0.588706,...,-0.522349,-0.256857,-0.843151,0.015503,1.901046,1.070732,-0.472406,-0.444554,-0.419852,0
2,-0.129229,0.601067,1.723404,1.172176,0.400545,2.093337,1.968770,2.852370,3.232767,3.066105,...,1.017585,0.836373,-0.197833,1.231812,2.827246,4.120162,1.309360,0.252761,0.257582,0
3,-0.835555,-0.648910,0.481740,-0.719414,-0.987079,-1.149364,-0.193816,-0.084747,-1.000852,-0.610469,...,-0.137365,-1.009341,0.557326,-0.111785,-0.161060,-0.488635,-0.549875,-0.639154,1.034640,0
4,2.050790,0.856537,0.111327,-0.312227,-0.292365,-0.672796,-0.013735,1.317299,1.510531,1.772220,...,-1.073812,-0.753780,-0.060532,0.241793,-1.174638,-0.107456,-0.487900,0.447361,0.576375,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,-0.456232,-0.116681,-0.705146,-0.779738,-0.647842,0.990954,1.314965,0.407323,0.463980,0.448504,...,-0.189390,-0.129077,1.230104,-0.847228,0.328253,-0.228741,0.550172,1.841992,1.831621,1
204,0.136733,-0.861801,-0.366036,0.054026,0.014392,-0.148740,-0.369029,-0.388465,-0.635067,0.053253,...,-0.761663,-0.200066,0.351373,-0.422934,-0.335815,-0.765856,-0.735798,-0.282388,0.038412,1
205,1.004381,0.160078,-0.673843,-0.531979,-0.723629,0.212502,0.064137,-0.200113,-0.442014,0.332912,...,0.268428,-1.108725,-0.801960,-0.437077,0.118548,1.070732,0.906526,-0.039138,-0.678871,1
206,0.049533,-0.095392,0.134804,0.148821,-1.055648,0.522865,0.401585,-0.264859,0.139685,0.202404,...,-0.501539,-0.867363,0.227802,-0.804798,-0.825128,-0.765856,-0.007598,-0.704020,-0.340154,1


In [6]:
normalized_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
count,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,...,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000
mean,1.708035e-17,6.832142e-17,-1.195625e-16,1.622634e-16,-1.793437e-16,2.049643e-16,1.024821e-16,3.416071e-17,-3.757678e-16,3.416071e-17,...,1.024821e-16,3.416071e-17,-1.451830e-16,2.775558e-17,-2.391250e-16,3.416071e-17,-1.110223e-16,1.345078e-16,7.686159e-17,0.533654
std,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,...,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,0.500070
min,-1.206158e+00,-1.150725e+00,-1.104253e+00,-1.036115e+00,-1.236093e+00,-1.600493e+00,-1.921613e+00,-1.522110e+00,-1.443689e+00,-1.468833e+00,...,-1.313126e+00,-1.449472e+00,-1.364897e+00,-1.229092e+00,-1.366868e+00,-1.302971e+00,-1.185113e+00,-1.271603e+00,-1.176985e+00,0.000000
25%,-6.894939e-01,-6.686781e-01,-6.490624e-01,-6.359298e-01,-6.703975e-01,-6.367565e-01,-6.626732e-01,-6.400918e-01,-6.856590e-01,-7.232644e-01,...,-6.394049e-01,-7.999231e-01,-7.642025e-01,-7.270112e-01,-6.678488e-01,-7.138771e-01,-6.738235e-01,-6.918580e-01,-6.788714e-01,0.000000
50%,-2.774703e-01,-2.322506e-01,-2.486515e-01,-2.120457e-01,-2.292089e-01,-2.106432e-01,-2.400524e-01,-2.672134e-01,-2.180558e-01,-1.928459e-01,...,-2.102002e-01,-1.645716e-01,-2.252935e-01,-2.532164e-01,-2.396997e-01,-3.240352e-01,-3.329639e-01,-2.499546e-01,-2.405314e-01,1.000000
75%,2.784345e-01,2.893335e-01,3.682681e-01,2.285353e-01,4.524231e-01,5.012417e-01,5.232608e-01,4.096773e-01,4.692723e-01,4.507410e-01,...,3.438640e-01,5.950106e-01,4.886751e-01,3.973675e-01,4.112618e-01,4.513169e-01,3.719959e-01,3.865486e-01,4.020352e-01,1.000000
max,4.706053e+00,5.944643e+00,6.836142e+00,8.025419e+00,5.878863e+00,4.710224e+00,4.074573e+00,3.816498e+00,4.274237e+00,3.746234e+00,...,5.980752e+00,4.016680e+00,3.330819e+00,5.008027e+00,5.448568e+00,4.795888e+00,5.585599e+00,4.615037e+00,7.450343e+00,1.000000


Train test val split
80/10/10

In [7]:
X = normalized_df.drop(columns=60, axis=1)
y = normalized_df[60]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [9]:
print(X_train.shape)
print(y_train.value_counts())
print('******')
print(X_test.shape)
print(y_test.value_counts())
print('******')
print(X_val.shape)
print(y_val.value_counts())

(166, 60)
1    91
0    75
Name: 60, dtype: int64
******
(21, 60)
1    11
0    10
Name: 60, dtype: int64
******
(21, 60)
0    12
1     9
Name: 60, dtype: int64


Tensorflow

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from wandb.keras import WandbCallback

2023-03-28 22:44:09.079692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Vytvorenie modelu

In [11]:
#nothing
tf_model1 = Sequential()
tf_model1.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(16, activation=tf.keras.activations.relu))
tf_model1.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout
tf_model2 = Sequential()
tf_model2.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dropout(0.5))
tf_model2.add(Dense(16, activation=tf.keras.activations.relu))
tf_model2.add(Dense(1, activation=tf.keras.activations.sigmoid))
#early stopping
tf_model3 = Sequential()
tf_model3.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(16, activation=tf.keras.activations.relu))
tf_model3.add(Dense(1, activation=tf.keras.activations.sigmoid))
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#regularization
tf_model4 = Sequential()
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.005)))
tf_model4.add(Dense(1, activation=tf.keras.activations.sigmoid))
#dropout + earlystop
tf_model5 = Sequential()
tf_model5.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dropout(0.5))
tf_model5.add(Dense(16, activation=tf.keras.activations.relu))
tf_model5.add(Dense(1, activation=tf.keras.activations.sigmoid))



2023-03-28 22:44:12.183471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
tf_model1.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model2.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model3.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model4.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])
tf_model5.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy', 'mse', precision, recall])

In [13]:
def fitModel(model, name, callbackArg):
    
    model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=100,
    validation_data=(X_val,y_val),
    callbacks=callbackArg
    )

    model.save('./'+ name)
    

In [14]:
fitModel(tf_model1, 'model1', [])

Epoch 1/100
11/11 [==============================] - 1s 17ms/step - loss: 0.6991 - accuracy: 0.5301 - mse: 0.2527 - precision: 0.5414 - recall: 0.9341 - val_loss: 0.7403 - val_accuracy: 0.4762 - val_mse: 0.2729 - val_precision: 0.4444 - val_recall: 0.8889
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.6562 - accuracy: 0.5904 - mse: 0.2325 - precision: 0.5793 - recall: 0.9231 - val_loss: 0.7027 - val_accuracy: 0.4762 - val_mse: 0.2551 - val_precision: 0.4444 - val_recall: 0.8889
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 0.6254 - accuracy: 0.6807 - mse: 0.2181 - precision: 0.6484 - recall: 0.9121 - val_loss: 0.6838 - val_accuracy: 0.5238 - val_mse: 0.2462 - val_precision: 0.4706 - val_recall: 0.8889
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5992 - accuracy: 0.7349 - mse: 0.2066 - precision: 0.6942 - recall: 0.9231 - val_loss: 0.6678 - val_accuracy: 0.5714 - val_mse: 0.2387 - val_precision: 0.5000 

11/11 [==============================] - 0s 2ms/step - loss: 0.0386 - accuracy: 1.0000 - mse: 0.0053 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5504 - val_accuracy: 0.8095 - val_mse: 0.1698 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 34/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0345 - accuracy: 1.0000 - mse: 0.0044 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5476 - val_accuracy: 0.8095 - val_mse: 0.1692 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 35/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0307 - accuracy: 1.0000 - mse: 0.0036 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5730 - val_accuracy: 0.8095 - val_mse: 0.1723 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 36/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0273 - accuracy: 1.0000 - mse: 0.0028 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5886 - val_accuracy: 0.8095 - val_mse: 0.1741 - val_precision: 0.7273 - val_reca

Epoch 65/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 1.0000 - mse: 4.1067e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8493 - val_accuracy: 0.8095 - val_mse: 0.1852 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 66/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 1.0000 - mse: 3.8483e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8598 - val_accuracy: 0.8095 - val_mse: 0.1855 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 67/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 1.0000 - mse: 3.4087e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8672 - val_accuracy: 0.8095 - val_mse: 0.1857 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 68/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 1.0000 - mse: 3.1281e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8718 - val_accuracy: 0.8095 - val_mse: 0.1858 - val

11/11 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000 - mse: 4.3444e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.0097 - val_accuracy: 0.8095 - val_mse: 0.1882 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 97/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000 - mse: 4.1269e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.0112 - val_accuracy: 0.8095 - val_mse: 0.1883 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 98/100
11/11 [==============================] - 0s 3ms/step - loss: 9.8176e-04 - accuracy: 1.0000 - mse: 3.8860e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.0166 - val_accuracy: 0.8095 - val_mse: 0.1884 - val_precision: 0.7273 - val_recall: 0.8889
Epoch 99/100
11/11 [==============================] - 0s 3ms/step - loss: 9.5720e-04 - accuracy: 1.0000 - mse: 3.6898e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.0216 - val_accuracy: 0.8095 - val_mse: 0.1884 - val_prec

In [15]:
fitModel(tf_model2, 'model2', [])

Epoch 1/100
11/11 [==============================] - 0s 15ms/step - loss: 0.8240 - accuracy: 0.5120 - mse: 0.2929 - precision: 0.5773 - recall: 0.5600 - val_loss: 0.7736 - val_accuracy: 0.4286 - val_mse: 0.2872 - val_precision: 0.4000 - val_recall: 0.6667
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.5663 - mse: 0.2573 - precision: 0.5979 - recall: 0.6374 - val_loss: 0.7591 - val_accuracy: 0.4286 - val_mse: 0.2806 - val_precision: 0.4000 - val_recall: 0.6667
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 0.8258 - accuracy: 0.5482 - mse: 0.2874 - precision: 0.5833 - recall: 0.6154 - val_loss: 0.7435 - val_accuracy: 0.4762 - val_mse: 0.2736 - val_precision: 0.4286 - val_recall: 0.6667
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 0.7220 - accuracy: 0.5843 - mse: 0.2569 - precision: 0.6100 - recall: 0.6703 - val_loss: 0.7361 - val_accuracy: 0.4762 - val_mse: 0.2702 - val_precision: 0.4286 

11/11 [==============================] - 0s 3ms/step - loss: 0.5387 - accuracy: 0.7108 - mse: 0.1838 - precision: 0.7216 - recall: 0.7692 - val_loss: 0.5810 - val_accuracy: 0.6667 - val_mse: 0.1991 - val_precision: 0.5714 - val_recall: 0.8889
Epoch 34/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5626 - accuracy: 0.6988 - mse: 0.1936 - precision: 0.6814 - recall: 0.8462 - val_loss: 0.5758 - val_accuracy: 0.6667 - val_mse: 0.1972 - val_precision: 0.5714 - val_recall: 0.8889
Epoch 35/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5274 - accuracy: 0.7530 - mse: 0.1759 - precision: 0.7404 - recall: 0.8462 - val_loss: 0.5664 - val_accuracy: 0.6667 - val_mse: 0.1938 - val_precision: 0.5714 - val_recall: 0.8889
Epoch 36/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5396 - accuracy: 0.6747 - mse: 0.1848 - precision: 0.6667 - recall: 0.8132 - val_loss: 0.5602 - val_accuracy: 0.6667 - val_mse: 0.1918 - val_precision: 0.5714 - val_reca

11/11 [==============================] - 0s 3ms/step - loss: 0.3967 - accuracy: 0.7892 - mse: 0.1332 - precision: 0.7800 - recall: 0.8571 - val_loss: 0.4604 - val_accuracy: 0.7143 - val_mse: 0.1634 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 66/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3766 - accuracy: 0.8193 - mse: 0.1250 - precision: 0.7961 - recall: 0.9011 - val_loss: 0.4558 - val_accuracy: 0.7143 - val_mse: 0.1621 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 67/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3401 - accuracy: 0.8072 - mse: 0.1114 - precision: 0.7980 - recall: 0.8681 - val_loss: 0.4576 - val_accuracy: 0.7143 - val_mse: 0.1636 - val_precision: 0.6364 - val_recall: 0.7778
Epoch 68/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3353 - accuracy: 0.8012 - mse: 0.1098 - precision: 0.7843 - recall: 0.8791 - val_loss: 0.4568 - val_accuracy: 0.7143 - val_mse: 0.1640 - val_precision: 0.6364 - val_reca

11/11 [==============================] - 0s 3ms/step - loss: 0.2942 - accuracy: 0.8434 - mse: 0.0956 - precision: 0.8736 - recall: 0.8352 - val_loss: 0.4792 - val_accuracy: 0.7619 - val_mse: 0.1572 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 98/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3053 - accuracy: 0.8193 - mse: 0.1033 - precision: 0.8506 - recall: 0.8132 - val_loss: 0.4838 - val_accuracy: 0.7619 - val_mse: 0.1577 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 99/100
11/11 [==============================] - 0s 3ms/step - loss: 0.2968 - accuracy: 0.8614 - mse: 0.0973 - precision: 0.9048 - recall: 0.8352 - val_loss: 0.4822 - val_accuracy: 0.7619 - val_mse: 0.1577 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 100/100
11/11 [==============================] - 0s 3ms/step - loss: 0.2787 - accuracy: 0.8434 - mse: 0.0918 - precision: 0.8916 - recall: 0.8132 - val_loss: 0.4842 - val_accuracy: 0.7619 - val_mse: 0.1586 - val_precision: 0.7000 - val_rec

In [16]:
fitModel(tf_model3, 'model3', [earlyStop])

Epoch 1/100
11/11 [==============================] - 0s 16ms/step - loss: 0.6999 - accuracy: 0.5482 - mse: 0.2521 - precision: 0.5575 - recall: 0.9700 - val_loss: 0.7369 - val_accuracy: 0.4286 - val_mse: 0.2707 - val_precision: 0.4286 - val_recall: 1.0000
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.6539 - accuracy: 0.5542 - mse: 0.2320 - precision: 0.5541 - recall: 0.9560 - val_loss: 0.6934 - val_accuracy: 0.5238 - val_mse: 0.2494 - val_precision: 0.4737 - val_recall: 1.0000
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 0.6163 - accuracy: 0.6325 - mse: 0.2144 - precision: 0.6056 - recall: 0.9451 - val_loss: 0.6716 - val_accuracy: 0.7143 - val_mse: 0.2380 - val_precision: 0.6000 - val_recall: 1.0000
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5866 - accuracy: 0.7048 - mse: 0.2005 - precision: 0.6694 - recall: 0.9121 - val_loss: 0.6460 - val_accuracy: 0.7143 - val_mse: 0.2254 - val_precision: 0.6000 

In [17]:
fitModel(tf_model4, 'model4', [])

Epoch 1/100
11/11 [==============================] - 1s 16ms/step - loss: 2.2075 - accuracy: 0.5120 - mse: 0.2532 - precision: 0.5455 - recall: 0.7800 - val_loss: 2.1692 - val_accuracy: 0.4286 - val_mse: 0.2505 - val_precision: 0.4211 - val_recall: 0.8889
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 2.1232 - accuracy: 0.5843 - mse: 0.2395 - precision: 0.5775 - recall: 0.9011 - val_loss: 2.1095 - val_accuracy: 0.4762 - val_mse: 0.2482 - val_precision: 0.4500 - val_recall: 1.0000
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 2.0489 - accuracy: 0.6024 - mse: 0.2299 - precision: 0.5874 - recall: 0.9231 - val_loss: 2.0478 - val_accuracy: 0.5238 - val_mse: 0.2442 - val_precision: 0.4737 - val_recall: 1.0000
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 1.9803 - accuracy: 0.6145 - mse: 0.2227 - precision: 0.5957 - recall: 0.9231 - val_loss: 1.9907 - val_accuracy: 0.6190 - val_mse: 0.2419 - val_precision: 0.5294 

11/11 [==============================] - 0s 3ms/step - loss: 0.6188 - accuracy: 0.9819 - mse: 0.0231 - precision: 1.0000 - recall: 0.9670 - val_loss: 1.0422 - val_accuracy: 0.8095 - val_mse: 0.1364 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 34/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5988 - accuracy: 0.9819 - mse: 0.0213 - precision: 1.0000 - recall: 0.9670 - val_loss: 1.0192 - val_accuracy: 0.8095 - val_mse: 0.1362 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 35/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5817 - accuracy: 0.9819 - mse: 0.0205 - precision: 1.0000 - recall: 0.9670 - val_loss: 1.0066 - val_accuracy: 0.8095 - val_mse: 0.1365 - val_precision: 0.7778 - val_recall: 0.7778
Epoch 36/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5641 - accuracy: 0.9880 - mse: 0.0192 - precision: 1.0000 - recall: 0.9780 - val_loss: 1.0008 - val_accuracy: 0.8095 - val_mse: 0.1362 - val_precision: 0.7778 - val_reca

11/11 [==============================] - 0s 3ms/step - loss: 0.3362 - accuracy: 1.0000 - mse: 0.0072 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8657 - val_accuracy: 0.7619 - val_mse: 0.1633 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 66/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3310 - accuracy: 1.0000 - mse: 0.0067 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8317 - val_accuracy: 0.7619 - val_mse: 0.1547 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 67/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3305 - accuracy: 1.0000 - mse: 0.0073 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8263 - val_accuracy: 0.7619 - val_mse: 0.1549 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 68/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3263 - accuracy: 1.0000 - mse: 0.0068 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.8478 - val_accuracy: 0.7619 - val_mse: 0.1606 - val_precision: 0.7000 - val_reca

11/11 [==============================] - 0s 3ms/step - loss: 0.2504 - accuracy: 1.0000 - mse: 0.0044 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.7793 - val_accuracy: 0.7619 - val_mse: 0.1535 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 98/100
11/11 [==============================] - 0s 3ms/step - loss: 0.2483 - accuracy: 1.0000 - mse: 0.0044 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.7735 - val_accuracy: 0.7619 - val_mse: 0.1520 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 99/100
11/11 [==============================] - 0s 3ms/step - loss: 0.2485 - accuracy: 1.0000 - mse: 0.0047 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.7980 - val_accuracy: 0.7619 - val_mse: 0.1541 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 100/100
11/11 [==============================] - 0s 3ms/step - loss: 0.2450 - accuracy: 1.0000 - mse: 0.0042 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.7930 - val_accuracy: 0.7619 - val_mse: 0.1570 - val_precision: 0.7000 - val_rec

In [18]:
fitModel(tf_model5, 'model5', [earlyStop])

Epoch 1/100
11/11 [==============================] - 0s 15ms/step - loss: 0.7575 - accuracy: 0.4699 - mse: 0.2750 - precision: 0.5361 - recall: 0.5200 - val_loss: 0.7099 - val_accuracy: 0.3810 - val_mse: 0.2582 - val_precision: 0.2500 - val_recall: 0.2222
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 0.7265 - accuracy: 0.4639 - mse: 0.2647 - precision: 0.5106 - recall: 0.5275 - val_loss: 0.7056 - val_accuracy: 0.3810 - val_mse: 0.2562 - val_precision: 0.2500 - val_recall: 0.2222
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 0.7372 - accuracy: 0.4277 - mse: 0.2706 - precision: 0.4783 - recall: 0.4835 - val_loss: 0.7033 - val_accuracy: 0.3810 - val_mse: 0.2550 - val_precision: 0.2500 - val_recall: 0.2222
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.5301 - mse: 0.2477 - precision: 0.5684 - recall: 0.5934 - val_loss: 0.7018 - val_accuracy: 0.4286 - val_mse: 0.2543 - val_precision: 0.3333 

11/11 [==============================] - 0s 3ms/step - loss: 0.5220 - accuracy: 0.7108 - mse: 0.1765 - precision: 0.6903 - recall: 0.8571 - val_loss: 0.5687 - val_accuracy: 0.6190 - val_mse: 0.1904 - val_precision: 0.5455 - val_recall: 0.6667
Epoch 34/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.7289 - mse: 0.1855 - precision: 0.7091 - recall: 0.8571 - val_loss: 0.5590 - val_accuracy: 0.6190 - val_mse: 0.1866 - val_precision: 0.5455 - val_recall: 0.6667
Epoch 35/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5269 - accuracy: 0.7349 - mse: 0.1818 - precision: 0.7156 - recall: 0.8571 - val_loss: 0.5537 - val_accuracy: 0.6190 - val_mse: 0.1840 - val_precision: 0.5455 - val_recall: 0.6667
Epoch 36/100
11/11 [==============================] - 0s 3ms/step - loss: 0.5202 - accuracy: 0.7169 - mse: 0.1763 - precision: 0.7157 - recall: 0.8022 - val_loss: 0.5547 - val_accuracy: 0.6190 - val_mse: 0.1838 - val_precision: 0.5455 - val_reca

11/11 [==============================] - 0s 3ms/step - loss: 0.3407 - accuracy: 0.8494 - mse: 0.1094 - precision: 0.8438 - recall: 0.8901 - val_loss: 0.5462 - val_accuracy: 0.7619 - val_mse: 0.1545 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 66/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3790 - accuracy: 0.8133 - mse: 0.1239 - precision: 0.8261 - recall: 0.8352 - val_loss: 0.5478 - val_accuracy: 0.7619 - val_mse: 0.1521 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 67/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3218 - accuracy: 0.8434 - mse: 0.1055 - precision: 0.8421 - recall: 0.8791 - val_loss: 0.5557 - val_accuracy: 0.7619 - val_mse: 0.1530 - val_precision: 0.7000 - val_recall: 0.7778
Epoch 68/100
11/11 [==============================] - 0s 3ms/step - loss: 0.3857 - accuracy: 0.8193 - mse: 0.1162 - precision: 0.8211 - recall: 0.8571 - val_loss: 0.5645 - val_accuracy: 0.7619 - val_mse: 0.1551 - val_precision: 0.7000 - val_reca

In [19]:
#pomocna funkcia
def predictEval(tf_model, XX, yy):
    # vykonanie predikcie
    y_pred = tf_model.predict(XX)
    # uprava outputu na boolean
    y_pred_bool = np.copy(y_pred)
    for x in y_pred_bool:
        x[0] = round(x[0])
    y_pred_bool

    #vratenie y a accuaracy
    return [y_pred, y_pred_bool, accuracy_score(y_pred_bool, yy), mean_squared_error(y_pred_bool, yy), precision_score(y_pred_bool, yy), recall_score(y_pred_bool, yy)]

Eval

In [20]:
def predictEvalWrap(model, name):
    train = predictEval(model, X_train, y_train)
    val = predictEval(model, X_val, y_val)
    test = predictEval(model, X_test, y_test)

    print(name)
    print('Accuracy score')
    print(f'Train: {train[2]*100:.2f}%')
    print(f'Val: {val[2]*100:.2f}%')
    print(f'Test: {test[2]*100:.2f}%')
    print('Mean squared error')
    print(f'Train: {train[3]*100:.2f}%')
    print(f'Val: {val[3]*100:.2f}%')
    print(f'Test: {test[3]*100:.2f}%')
    print('Precision')
    print(f'Train: {train[4]*100:.2f}%')
    print(f'Val: {val[4]*100:.2f}%')
    print(f'Test: {test[4]*100:.2f}%')
    print('Recall')
    print(f'Train: {train[5]*100:.2f}%')
    print(f'Val: {val[5]*100:.2f}%')
    print(f'Test: {test[5]*100:.2f}%')

In [21]:
predictEvalWrap(tf_model1, 'No overfit prevention')
predictEvalWrap(tf_model2, 'Dropout')
predictEvalWrap(tf_model3, 'Early stopping')
predictEvalWrap(tf_model4, 'Regularization')
predictEvalWrap(tf_model5, 'Dropout + Early stopping')

1/1 [==============================] - 0s 13ms/step
No overfit prevention
Accuracy score
Train: 100.00%
Val: 80.95%
Test: 85.71%
Mean squared error
Train: 0.00%
Val: 19.05%
Test: 14.29%
Precision
Train: 100.00%
Val: 88.89%
Test: 100.00%
Recall
Train: 100.00%
Val: 72.73%
Test: 78.57%
1/1 [==============================] - 0s 12ms/step
Dropout
Accuracy score
Train: 96.39%
Val: 76.19%
Test: 80.95%
Mean squared error
Train: 3.61%
Val: 23.81%
Test: 19.05%
Precision
Train: 97.80%
Val: 77.78%
Test: 81.82%
Recall
Train: 95.70%
Val: 70.00%
Test: 81.82%
1/1 [==============================] - 0s 12ms/step
Early stopping
Accuracy score
Train: 99.40%
Val: 71.43%
Test: 76.19%
Mean squared error
Train: 0.60%
Val: 28.57%
Test: 23.81%
Precision
Train: 98.90%
Val: 77.78%
Test: 81.82%
Recall
Train: 100.00%
Val: 63.64%
Test: 75.00%
1/1 [==============================] - 0s 12ms/step
Regularization
Accuracy score
Train: 100.00%
Val: 76.19%
Test: 76.19%
Mean squared error
Train: 0.00%
Val: 23.81%
Test: 23.8

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

In [23]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [24]:
class PyTorchModelDropout(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.5):
        super(PyTorchModelDropout, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(16, 16)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(16, 16)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = torch.sigmoid(self.fc4(x))
        return x

In [25]:
class SonarDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [26]:
train_dataset = SonarDataset(X_train, y_train)
val_dataset = SonarDataset(X_val, y_val)
test_dataset = SonarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [27]:
# Instantiate the model
input_dim = len(X_train.columns)
model = PyTorchModel(input_dim)
model_dropout = PyTorchModelDropout(input_dim, dropout_rate=0.5)
model_early_stopping = PyTorchModel(input_dim)

In [28]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
optimizer_dropout = optim.Adam(model_dropout.parameters())
optimizer_early_stopping = optim.Adam(model_early_stopping.parameters())

In [29]:
num_epochs = 100

In [30]:

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

Epoch: 1/100, Loss: 0.6699, Val Loss: 0.7054
Epoch: 2/100, Loss: 0.6647, Val Loss: 0.7053
Epoch: 3/100, Loss: 0.6119, Val Loss: 0.7057
Epoch: 4/100, Loss: 0.6692, Val Loss: 0.7072
Epoch: 5/100, Loss: 0.7123, Val Loss: 0.7063
Epoch: 6/100, Loss: 0.5804, Val Loss: 0.6995
Epoch: 7/100, Loss: 0.6390, Val Loss: 0.6928
Epoch: 8/100, Loss: 0.4733, Val Loss: 0.6733
Epoch: 9/100, Loss: 0.3155, Val Loss: 0.6403
Epoch: 10/100, Loss: 0.4065, Val Loss: 0.6010
Epoch: 11/100, Loss: 0.3615, Val Loss: 0.5622
Epoch: 12/100, Loss: 0.3025, Val Loss: 0.5231
Epoch: 13/100, Loss: 0.1330, Val Loss: 0.5041
Epoch: 14/100, Loss: 0.1786, Val Loss: 0.4923
Epoch: 15/100, Loss: 0.3210, Val Loss: 0.5097
Epoch: 16/100, Loss: 0.0179, Val Loss: 0.5094
Epoch: 17/100, Loss: 0.1601, Val Loss: 0.5293
Epoch: 18/100, Loss: 0.0622, Val Loss: 0.5465
Epoch: 19/100, Loss: 0.3940, Val Loss: 0.5480
Epoch: 20/100, Loss: 0.0724, Val Loss: 0.5812
Epoch: 21/100, Loss: 0.2850, Val Loss: 0.6137
Epoch: 22/100, Loss: 0.1049, Val Loss: 0.61

In [31]:
for epoch in range(num_epochs):
    model_dropout.train()
    for X_batch, y_batch in train_loader:
        optimizer_dropout.zero_grad()
        y_pred = model_dropout(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        model_dropout.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_dropout(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count
    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

Epoch: 1/100, Loss: 0.6915, Val Loss: 0.6941
Epoch: 2/100, Loss: 0.6863, Val Loss: 0.6941
Epoch: 3/100, Loss: 0.6853, Val Loss: 0.6941
Epoch: 4/100, Loss: 0.7485, Val Loss: 0.6941
Epoch: 5/100, Loss: 0.6858, Val Loss: 0.6941
Epoch: 6/100, Loss: 0.6753, Val Loss: 0.6941
Epoch: 7/100, Loss: 0.6941, Val Loss: 0.6941
Epoch: 8/100, Loss: 0.6825, Val Loss: 0.6941
Epoch: 9/100, Loss: 0.6948, Val Loss: 0.6941
Epoch: 10/100, Loss: 0.6802, Val Loss: 0.6941
Epoch: 11/100, Loss: 0.7008, Val Loss: 0.6941
Epoch: 12/100, Loss: 0.6922, Val Loss: 0.6941
Epoch: 13/100, Loss: 0.6346, Val Loss: 0.6941
Epoch: 14/100, Loss: 0.6850, Val Loss: 0.6941
Epoch: 15/100, Loss: 0.7035, Val Loss: 0.6941
Epoch: 16/100, Loss: 0.6729, Val Loss: 0.6941
Epoch: 17/100, Loss: 0.7157, Val Loss: 0.6941
Epoch: 18/100, Loss: 0.6800, Val Loss: 0.6941
Epoch: 19/100, Loss: 0.6833, Val Loss: 0.6941
Epoch: 20/100, Loss: 0.6715, Val Loss: 0.6941
Epoch: 21/100, Loss: 0.7234, Val Loss: 0.6941
Epoch: 22/100, Loss: 0.6930, Val Loss: 0.69

In [32]:
patience = 10

best_val_loss = float("inf")
counter = 0

for epoch in range(num_epochs):
    model_early_stopping.train()
    for X_batch, y_batch in train_loader:
        optimizer_early_stopping.zero_grad()
        y_pred = model_early_stopping(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        model_early_stopping.eval()
        val_loss_sum = 0
        val_batch_count = 0
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model_early_stopping(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_loss_sum += val_loss.item()
            val_batch_count += 1
        val_loss_avg = val_loss_sum / val_batch_count

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}")

    # Early stopping logic
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping!")
            break

Epoch: 1/100, Loss: 0.6456, Val Loss: 0.7059
Epoch: 2/100, Loss: 0.6719, Val Loss: 0.7059
Epoch: 3/100, Loss: 0.6455, Val Loss: 0.7059
Epoch: 4/100, Loss: 0.6965, Val Loss: 0.7059
Epoch: 5/100, Loss: 0.6986, Val Loss: 0.7059
Epoch: 6/100, Loss: 0.6428, Val Loss: 0.7059
Epoch: 7/100, Loss: 0.6971, Val Loss: 0.7059
Epoch: 8/100, Loss: 0.6970, Val Loss: 0.7059
Epoch: 9/100, Loss: 0.6705, Val Loss: 0.7059
Epoch: 10/100, Loss: 0.6937, Val Loss: 0.7059
Epoch: 11/100, Loss: 0.6694, Val Loss: 0.7059
Early stopping!


In [33]:
def predict_eval(model, loader):
    with torch.no_grad():
        model.eval()
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            y_pred_bool = torch.round(y_pred)
            accuracy = accuracy_score(y_pred_bool, y_batch)
    return accuracy

In [34]:
train_accuracy = predict_eval(model, train_loader)
val_accuracy = predict_eval(model, val_loader)
test_accuracy = predict_eval(model, test_loader)

In [35]:
train_accuracy_dropout = predict_eval(model_dropout, train_loader)
val_accuracy_dropout = predict_eval(model_dropout, val_loader)
test_accuracy_dropout = predict_eval(model_dropout, test_loader)

In [36]:
train_accuracy_early_stopping = predict_eval(model_early_stopping, train_loader)
val_accuracy_early_stopping = predict_eval(model_early_stopping, val_loader)
test_accuracy_early_stopping = predict_eval(model_early_stopping, test_loader)

In [37]:
print('No overfit prevention')
print(f'Train: {train_accuracy * 100:.2f}%')
print(f'Val: {val_accuracy * 100:.2f}%')
print(f'Test: {test_accuracy * 100:.2f}%')

No overfit prevention
Train: 100.00%
Val: 66.67%
Test: 71.43%


In [38]:
print('Droupout')
print(f'Train: {train_accuracy_dropout * 100:.2f}%')
print(f'Val: {val_accuracy_dropout * 100:.2f}%')
print(f'Test: {test_accuracy_dropout * 100:.2f}%')

Droupout
Train: 33.33%
Val: 57.14%
Test: 47.62%


In [39]:
print('Early stopping')
print(f'Train: {train_accuracy_early_stopping * 100:.2f}%')
print(f'Val: {val_accuracy_early_stopping * 100:.2f}%')
print(f'Test: {test_accuracy_early_stopping * 100:.2f}%')

Early stopping
Train: 16.67%
Val: 42.86%
Test: 52.38%
